In [27]:
#dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import gmaps.datasets
from scipy.stats import linregress
from scipy import stats
import os

In [28]:
crime_path = os.path.join ("../Resources/cd_33613.csv")
crimes_data = pd.read_csv(crime_path)
propreties_path = os.path.join ("../../GB/Resources/All_Hill_Prop_Value.csv")
propreties_data = pd.read_csv(propreties_path)
schools_path= os.path.join ("../../KS/Resources/school_grades_final.csv")
schools_data = pd.read_csv(schools_path)

In [29]:
crimes_data.head()

,Unnamed: 0,Reported Date,Case Number,Crime Type,Crime Type Extension,zip,year
0,0,1/1/2016,2016-281,DUI,DUI (DRIVING UNDER INFLUENCE),33613,2016
1,6,1/1/2016,2016-60,BATTERY,BATTERY-SIMPLE,33613,2016
2,7,1/2/2016,2016-2990,BURGLARY,THEFT FROM A VEHICLE,33613,2016
3,9,1/2/2016,2016-3345,THEFT,THEFT FROM A BUILDING,33613,2016
4,10,1/2/2016,2016-3552,BATTERY,BATTERY-SIMPLE,33613,2016


In [30]:
propreties_data.head()

,Unnamed: 0,SEQ_NO,PARCEL_ID,PropertyValue,NBRHD_CD,CENSUS_BK,PropAddress,City,ZipCode
0,0,1,153323ZZZ000000000200A,4628604,22700200,120579900000,EGMONT KEY,ST PETERSBURG,33715
1,3,4,172701001000000000001U,147699,21100700,120570115091,19507 PINE TREE RD,ODESSA,33556
2,4,5,172701001000000000010U,140829,21100700,120570115092,19931 ANGEL LN,ODESSA,33556
3,5,6,172701001000000000011U,462229,21100700,120570115092,19859 ANGEL LN,ODESSA,33556
4,6,7,172701001000000000012U,496434,21100700,120570115091,7320 W COUNTY LINE RD,ODESSA,33556


In [31]:
schools_data.head()

,Unnamed: 0,District Number,District Name,School Number,School Name,Total Points Earned,Total Components,Percent of Total Possible Points,Percent Tested,Grade 2019,...,Grade 2017,Grade 2016,Informational Baseline Grade 2015,Charter School,Title I,Alternative/ESE Center School,School Type,Percent of Minority Students,Percent of Economically Disadvantaged Students,ZIP
0,1420,29,HILLSBOROUGH,42,FOREST HILLS ELEMENTARY SCHOOL,338,7,48,100,C,...,D,F,F,NO,YES,N,1,85.6,94.6,33612
1,1422,29,HILLSBOROUGH,51,SHEEHY ELEMENTARY SCHOOL,351,7,50,100,C,...,D,D,F,NO,YES,N,1,97.4,98.0,33610
2,1424,29,HILLSBOROUGH,54,CORR ELEMENTARY SCHOOL,315,7,45,100,C,...,C,C,D,NO,YES,N,1,79.1,78.0,33534
3,1426,29,HILLSBOROUGH,56,DAVIS ELEMENTARY SCHOOL,333,7,48,100,C,...,C,C,C,NO,YES,N,1,85.8,90.5,33615
4,1427,29,HILLSBOROUGH,59,FISHHAWK CREEK ELEMENTARY SCHOOL,497,7,71,100,A,...,A,A,A,NO,NO,N,1,30.6,13.8,33547
